# Collecting data from api

In [1]:
import pandas as pd
from Data.APICalls import get_intervals, get_laps, get_pit_data, get_position, get_session, get_stints, get_weather
import numpy as np
import time
import random
import concurrent.futures

## Reading Drivers data

In [2]:
driver_df = pd.read_csv("../Data/drivers.csv")
meeting_keys = driver_df['meeting_key']
driver_numbers = driver_df['driver_number']

### Using Driver data to complete api calls

Sessions Data

In [3]:
session_type = ['Practice 1', 'Practice 2', 'Practice 3', 'Qualifying', 'Race']
year = [2022, 2023, 2024]

all_session_dfs = []

for s_type in session_type:
    session_dfs = []
    for y in year:
        session_df = get_session(s_type, y)
        session_dfs.append(session_df)
        time.sleep(1)

    session_df_concat = pd.concat(session_dfs, ignore_index=True)
    all_session_dfs.append(session_df_concat)

final_session_df = pd.concat(all_session_dfs, ignore_index=True)
final_session_df

,location,country_key,country_code,country_name,circuit_key,circuit_short_name,session_type,session_name,date_start,date_end,gmt_offset,session_key,meeting_key,year
0,Sakhir,36,BRN,Bahrain,63,Sakhir,Practice,Practice 1,2023-02-23T07:00:00+00:00,2023-02-23T16:30:00+00:00,03:00:00,9222,1140,2023
1,Sakhir,36,BRN,Bahrain,63,Sakhir,Practice,Practice 1,2023-03-03T11:30:00+00:00,2023-03-03T12:30:00+00:00,03:00:00,7765,1141,2023
2,Jeddah,153,KSA,Saudi Arabia,149,Jeddah,Practice,Practice 1,2023-03-17T13:30:00+00:00,2023-03-17T14:30:00+00:00,03:00:00,7772,1142,2023
3,Melbourne,5,AUS,Australia,10,Melbourne,Practice,Practice 1,2023-03-31T01:30:00+00:00,2023-03-31T02:30:00+00:00,11:00:00,7780,1143,2023
4,Baku,30,AZE,Azerbaijan,144,Baku,Practice,Practice 1,2023-04-28T09:30:00+00:00,2023-04-28T10:30:00+00:00,04:00:00,9063,1207,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,Monaco,114,MON,Monaco,22,Monte Carlo,Race,Race,2024-05-26T13:00:00+00:00,2024-05-26T15:00:00+00:00,02:00:00,9523,1236,2024
150,Montréal,46,CAN,Canada,23,Montreal,Race,Race,2024-06-09T18:00:00+00:00,2024-06-09T20:00:00+00:00,-04:00:00,9531,1237,2024
151,Barcelona,1,ESP,Spain,15,Catalunya,Race,Race,2024-06-23T13:00:00+00:00,2024-06-23T15:00:00+00:00,02:00:00,9539,1238,2024
152,Spielberg,17,AUT,Austria,19,Spielberg,Race,Race,2024-06-30T13:00:00+00:00,2024-06-30T15:00:00+00:00,02:00:00,9550,1239,2024


All session keys

In [4]:
session_keys = final_session_df['session_key'].tolist()
session_keys.append(9550)
session_keys

[9222,
 7765,
 7772,
 7780,
 9063,
 9071,
 9087,
 9095,
 9103,
 9119,
 9223,
 9134,
 9142,
 9150,
 9158,
 9166,
 9214,
 9206,
 9174,
 9198,
 9182,
 9190,
 9465,
 9473,
 9481,
 9489,
 9663,
 9497,
 9508,
 9516,
 9524,
 9532,
 9540,
 9551,
 7763,
 7766,
 7773,
 7781,
 9072,
 9088,
 9096,
 9104,
 9120,
 9127,
 9143,
 9151,
 9159,
 9167,
 9175,
 9183,
 9191,
 9466,
 9474,
 9482,
 9490,
 9509,
 9517,
 9525,
 9533,
 9552,
 7764,
 7767,
 7774,
 7782,
 9073,
 9089,
 9097,
 9105,
 9121,
 9128,
 9144,
 9152,
 9160,
 9168,
 9176,
 9184,
 9192,
 9467,
 9475,
 9483,
 9491,
 9510,
 9518,
 9526,
 9534,
 9553,
 7768,
 7775,
 7783,
 9064,
 9074,
 9090,
 9098,
 9106,
 9112,
 9122,
 9129,
 9135,
 9145,
 9153,
 9161,
 9169,
 9215,
 9207,
 9177,
 9304,
 9314,
 9193,
 9468,
 9476,
 9484,
 9492,
 9664,
 9498,
 9511,
 9519,
 9527,
 9535,
 9541,
 9554,
 7953,
 7779,
 7787,
 9070,
 9078,
 9094,
 9102,
 9110,
 9118,
 9126,
 9133,
 9141,
 9149,
 9157,
 9165,
 9173,
 9221,
 9213,
 9181,
 9205,
 9189,
 9197,
 9472,

In [7]:
final_session_df.to_csv("../Data/session_data.csv", index=False)

Weather data

In [8]:
meeting_key = 1239
randomWindDirection = random.randint(250, 330)
randomTrackTemperature = random.randint(10, 40)
final_weather_dfs = []

for i in range(20):
    weather_df = get_weather(meeting_key, randomWindDirection, randomTrackTemperature)
    final_weather_dfs.append(weather_df)
    time.sleep(1)
    
final_weather_df = pd.concat(final_weather_dfs, ignore_index=True)
final_weather_df

,meeting_key,session_key,date,air_temperature,humidity,pressure,rainfall,track_temperature,wind_direction,wind_speed
0,1239,9545,2024-06-28T14:21:19.297000+00:00,27.2,42.0,940.3,0,41.0,348,0.6
1,1239,9545,2024-06-28T14:23:19.293000+00:00,27.3,44.0,940.3,0,40.7,314,0.6
2,1239,9545,2024-06-28T14:25:19.302000+00:00,27.5,43.0,940.3,0,43.0,346,0.7
3,1239,9545,2024-06-28T14:26:19.295000+00:00,27.5,42.0,940.3,0,43.7,333,1.3
4,1239,9545,2024-06-28T14:49:19.399000+00:00,27.6,42.0,940.1,0,43.4,294,0.8
...,...,...,...,...,...,...,...,...,...,...
3055,1239,9550,2024-06-30T14:23:31.424000+00:00,29.9,34.0,932.8,0,45.7,292,2.7
3056,1239,9550,2024-06-30T14:26:31.427000+00:00,29.6,35.0,932.7,0,43.8,284,1.1
3057,1239,9550,2024-06-30T14:28:31.422000+00:00,29.6,34.0,932.8,0,43.5,329,1.0
3058,1239,9550,2024-06-30T14:29:31.430000+00:00,29.4,34.0,932.7,0,43.5,322,2.3


In [10]:
final_weather_df.to_csv("../Data/weather_data.csv", index=False)

Lap Data

In [11]:
driver_arrays = {driver_number: [] for driver_number in driver_numbers}

# Define a function to fetch lap data for a driver and lap
def fetch_lap_data(driver_number, lap, sess_key):
    lap_df = get_laps(sess_key, driver_number, lap)
    if lap_df.empty:
        raise ValueError(f"Driver {driver_number} lap {lap} is empty")
    return driver_number, lap, lap_df

# Use ThreadPoolExecutor for parallel processing
with concurrent.futures.ThreadPoolExecutor(max_workers=30) as executor:
    future_to_lap = {executor.submit(fetch_lap_data, driver_number, lap, 9550): (driver_number, lap)
                     for driver_number in driver_numbers for lap in range(2, 71)}
    for future in concurrent.futures.as_completed(future_to_lap):
        driver_number, lap = future_to_lap[future]
        try:
            driver_number, lap, lap_df = future.result()
            driver_arrays[driver_number].append(lap_df)
        except Exception as exc:
            print(f"Driver {driver_number} lap {lap} generated an exception: {exc}")

# Concatenate the DataFrames for each driver
for driver_number, driver_list in driver_arrays.items():
    driver_arrays[driver_number] = pd.concat(driver_list, ignore_index=True)

driver_arrays

HTTP 429: Too Many Requests. Retrying in 1 seconds...
HTTP 429: Too Many Requests. Retrying in 1 seconds...
HTTP 429: Too Many Requests. Retrying in 1 seconds...
HTTP 429: Too Many Requests. Retrying in 1 seconds...
HTTP 429: Too Many Requests. Retrying in 1 seconds...
HTTP 429: Too Many Requests. Retrying in 1 seconds...
HTTP 429: Too Many Requests. Retrying in 1 seconds...
HTTP 429: Too Many Requests. Retrying in 1 seconds...
HTTP 429: Too Many Requests. Retrying in 1 seconds...
HTTP 429: Too Many Requests. Retrying in 1 seconds...
HTTP 429: Too Many Requests. Retrying in 1 seconds...
HTTP 429: Too Many Requests. Retrying in 1 seconds...
HTTP 429: Too Many Requests. Retrying in 1 seconds...
HTTP 429: Too Many Requests. Retrying in 1 seconds...
HTTP 429: Too Many Requests. Retrying in 1 seconds...
HTTP 429: Too Many Requests. Retrying in 1 seconds...
HTTP 429: Too Many Requests. Retrying in 1 seconds...
HTTP 429: Too Many Requests. Retrying in 1 seconds...
HTTP 429: Too Many Requests.

{1:     meeting_key  session_key  driver_number i1_speed  i2_speed  st_speed  \
 0          1239         9550              1      295       228       290   
 1          1239         9550              1      297       230       289   
 2          1239         9550              1      290       227       286   
 3          1239         9550              1      294       225       297   
 4          1239         9550              1     None       229       289   
 ..          ...          ...            ...      ...       ...       ...   
 57         1239         9550              1      290       238       296   
 58         1239         9550              1     None       231       304   
 59         1239         9550              1      291       234       301   
 60         1239         9550              1      289       231       304   
 61         1239         9550              1      286       213       290   
 
                           date_start  lap_duration  is_pit_out_lap  \


In [12]:
for driver_number, df in driver_arrays.items():
    print(f"Driver {driver_number} has {df.shape[0]} rows")
    display(df)
    "\n"

Driver 1 has 62 rows


,meeting_key,session_key,driver_number,i1_speed,i2_speed,st_speed,date_start,lap_duration,is_pit_out_lap,duration_sector_1,duration_sector_2,duration_sector_3,segments_sector_1,segments_sector_2,segments_sector_3,lap_number
0,1239,9550,1,295,228,290,2024-06-30T13:13:35.778000+00:00,70.596,False,17.785,31.451,21.360,"[2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048]",10
1,1239,9550,1,297,230,289,2024-06-30T13:15:56.834000+00:00,70.512,False,17.697,31.454,21.361,"[None, 2048, 2048, 2048, 2049, 2049]","[2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2049, 2048, 2048, 2048, 2048]",12
2,1239,9550,1,290,227,286,2024-06-30T13:04:14.597000+00:00,69.938,False,17.697,31.186,21.055,"[2049, 2049, 2048, 2049, 2049, 2048]","[2048, 2049, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2049, 2048, 2048, 2048, 2048]",2
3,1239,9550,1,294,225,297,2024-06-30T13:23:02.059000+00:00,71.364,False,17.815,31.854,21.695,"[2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2049, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048]",18
4,1239,9550,1,None,229,289,2024-06-30T13:08:54.405000+00:00,70.238,False,17.698,31.295,21.245,"[2048, 2048, 2048, 2048, 2049, 2049]","[2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2049, 2048, 2048, 2048, 2048]",6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,1239,9550,1,290,238,296,2024-06-30T14:25:36.189000+00:00,68.072,False,17.214,30.512,20.346,"[None, 2048, 2048, 2048, 2048, 2048]","[2048, 2051, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048]",70
58,1239,9550,1,None,231,304,2024-06-30T14:10:33.697000+00:00,69.183,False,17.396,31.183,20.604,"[None, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048]",58
59,1239,9550,1,291,234,301,2024-06-30T14:04:49.468000+00:00,68.772,False,17.348,30.775,20.649,"[2048, 2048, 2051, 2049, 2049, 2048]","[2048, 2049, 2048, 2048, 2049, 2049, 2048]","[2049, 2049, 2048, 2051, 2049, 2048, 2048]",53
60,1239,9550,1,289,231,304,2024-06-30T14:12:53.676000+00:00,69.399,False,17.449,31.233,20.717,"[None, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2049, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048]",60


Driver 2 has 60 rows


,meeting_key,session_key,driver_number,i1_speed,i2_speed,st_speed,date_start,lap_duration,is_pit_out_lap,duration_sector_1,duration_sector_2,duration_sector_3,segments_sector_1,segments_sector_2,segments_sector_3,lap_number
0,1239,9550,2,297,226,290,2024-06-30T13:08:30.082000+00:00,71.895,False,17.915,32.013,21.967,"[2048, 2048, 2048, 2049, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2049, 2048]","[2049, 2048, 2048, 2049, 2048, 2048, 2048]",5
1,1239,9550,2,293,225,287,2024-06-30T13:07:18.217000+00:00,71.760,False,17.900,31.806,22.054,"[None, 2048, 2048, 2049, 2048, 2048]","[2048, 2049, 2048, 2048, 2049, 2048, 2048]","[2048, 2048, 2049, 2048, 2049, 2048, 2048]",4
2,1239,9550,2,299,223,286,2024-06-30T13:04:27.184000+00:00,99.346,True,45.278,32.073,21.995,"[2064, 2064, 2064, 2048, 2048, 2048]","[2049, 2049, 2049, 2048, 2049, 2049, 2049]","[2049, 2048, 2048, 2049, 2048, 2048, 2048]",2
3,1239,9550,2,297,226,291,2024-06-30T13:15:42.649000+00:00,72.332,False,17.914,32.366,22.052,"[None, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048]",11
4,1239,9550,2,296,222,293,2024-06-30T13:25:26.461000+00:00,72.772,False,17.910,32.602,22.260,"[2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2064, 2064, 2064]",19
5,1239,9550,2,296,225,289,2024-06-30T13:13:17.935000+00:00,72.297,False,17.863,32.261,22.173,"[2049, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048]",9
6,1239,9550,2,295,227,289,2024-06-30T13:06:06.548000+00:00,71.686,False,17.870,31.733,22.083,"[2049, 2049, 2049, 2049, 2048, 2048]","[2048, 2049, 2049, 2048, 2049, 2049, 2049]","[2048, 2048, 2048, 2049, 2049, 2048, 2048]",3
7,1239,9550,2,294,222,287,2024-06-30T13:18:07.623000+00:00,72.669,False,17.977,32.428,22.264,"[None, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048]",13
8,1239,9550,2,295,225,287,2024-06-30T13:20:33.087000+00:00,72.561,False,18.063,32.367,22.131,"[None, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048]",15
9,1239,9550,2,298,225,306,2024-06-30T13:37:50.604000+00:00,72.662,False,18.387,32.444,21.831,"[None, 2048, 2048, 2048, 2049, 2048]","[2048, 2048, 2048, 2049, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048]",29


Driver 3 has 55 rows


...

,meeting_key,session_key,driver_number,i1_speed,i2_speed,st_speed,date_start,lap_duration,is_pit_out_lap,duration_sector_1,duration_sector_2,duration_sector_3,segments_sector_1,segments_sector_2,segments_sector_3,lap_number
0,1239,9550,63,295,225,298,2024-06-30T13:04:16.930000+00:00,70.580,False,17.469,31.731,21.380,"[2049, 2051, 2049, 2049, 2048, 2048]","[2048, 2049, 2048, 2048, 2049, 2048, 2048]","[2049, 2049, 2048, 2049, 2049, 2048, 2048]",2
1,1239,9550,63,287,224,308,2024-06-30T13:05:27.617000+00:00,70.441,False,17.801,31.483,21.157,"[2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2049, 2049, 2049, 2048, 2049]","[2049, 2048, 2051, 2048, 2048, 2048, 2048]",3
2,1239,9550,63,294,224,294,2024-06-30T13:07:48.445000+00:00,70.193,False,17.610,31.332,21.251,"[2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2049, 2049]","[2048, 2048, 2048, 2048, 2048, 2048, 2048]",5
3,1239,9550,63,298,219,285,2024-06-30T13:08:58.691000+00:00,71.033,False,17.616,31.962,21.455,"[None, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048]",6
4,1239,9550,63,None,230,294,2024-06-30T13:10:09.738000+00:00,70.682,False,17.624,31.646,21.412,"[None, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2049]",7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,1239,9550,63,179,194,260,2024-06-30T14:20:09.857000+00:00,86.932,False,18.084,44.858,23.990,"[None, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2049, 2048, 2048, 2048, 2048, 2048]",66
61,1239,9550,63,299,235,309,2024-06-30T14:23:55.882000+00:00,69.747,False,17.500,31.450,20.797,"[None, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2049, 2048, 2048, 2048, 2048]",69
62,1239,9550,63,294,236,305,2024-06-30T14:22:46.143000+00:00,69.645,False,17.449,31.317,20.879,"[None, 2049, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2049, 2049, 2048, 2048, 2048, 2048]",68
63,1239,9550,63,301,231,307,2024-06-30T14:25:05.586000+00:00,69.627,False,17.452,31.391,20.784,"[None, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2049, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048]",70


Driver 77 has 65 rows


,meeting_key,session_key,driver_number,i1_speed,i2_speed,st_speed,date_start,lap_duration,is_pit_out_lap,duration_sector_1,duration_sector_2,duration_sector_3,segments_sector_1,segments_sector_2,segments_sector_3,lap_number
0,1239,9550,77,291,222,290,2024-06-30T13:13:59.321000+00:00,71.756,False,17.957,32.066,21.733,"[None, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048]",10
1,1239,9550,77,293,223,290,2024-06-30T13:15:11.065000+00:00,71.821,False,17.941,32.075,21.805,"[None, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2049, 2048, 2048, 2048, 2048]",11
2,1239,9550,77,293,224,289,2024-06-30T13:18:46.873000+00:00,72.120,False,18.043,32.243,21.834,"[None, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2049, 2048, 2048, 2048, 2048]",14
3,1239,9550,77,295,222,291,2024-06-30T13:16:22.762000+00:00,71.965,False,17.996,32.172,21.797,"[None, 2048, 2048, 2048, 2048, 2048]","[2049, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048]",12
4,1239,9550,77,294,222,290,2024-06-30T13:21:11.275000+00:00,72.754,False,18.030,32.474,22.250,"[None, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2049, 2048, 2048, 2048, 2048]",16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,1239,9550,77,293,228,303,2024-06-30T14:24:11.523000+00:00,71.085,False,17.692,31.910,21.483,"[None, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048]",68
61,1239,9550,77,302,226,310,2024-06-30T13:55:22.831000+00:00,71.193,False,17.514,32.088,21.591,"[2048, 2048, 2048, 2049, 2048, 2048]","[2048, 2048, 2048, 2049, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048]",44
62,1239,9550,77,296,228,303,2024-06-30T14:26:33.713000+00:00,71.022,False,17.692,31.815,21.515,"[None, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2064, 2064, 2064]",70
63,1239,9550,77,275,223,309,2024-06-30T14:11:58.009000+00:00,72.808,False,18.603,32.516,21.689,"[None, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048]",58


Driver 81 has 59 rows


,meeting_key,session_key,driver_number,i1_speed,i2_speed,st_speed,date_start,lap_duration,is_pit_out_lap,duration_sector_1,duration_sector_2,duration_sector_3,segments_sector_1,segments_sector_2,segments_sector_3,lap_number
0,1239,9550,81,None,224,289,2024-06-30T13:14:58.375000+00:00,71.394,False,17.872,31.964,21.558,"[None, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048]",11
1,1239,9550,81,293,226,291,2024-06-30T13:17:20.873000+00:00,71.300,False,17.967,31.856,21.477,"[None, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048]",13
2,1239,9550,81,297,227,291,2024-06-30T13:16:09.781000+00:00,71.153,False,17.931,31.826,21.396,"[None, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048]",12
3,1239,9550,81,294,226,289,2024-06-30T13:18:32.206000+00:00,71.424,False,17.907,31.828,21.689,"[None, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048]",14
4,1239,9550,81,None,227,306,2024-06-30T13:04:19.820000+00:00,70.541,False,17.600,31.425,21.516,"[2049, 2049, 2049, 2049, 2049, 2049]","[2048, 2049, 2049, 2049, 2049, 2049, 2049]","[2048, 2049, 2048, 2048, 2048, 2048, 2049]",2
5,1239,9550,81,314,227,304,2024-06-30T13:09:02.222000+00:00,71.246,False,17.558,31.897,21.791,"[2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2049, 2048, 0]",6
6,1239,9550,81,309,225,302,2024-06-30T13:06:40.833000+00:00,70.810,False,17.539,31.572,21.699,"[None, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2049, 2048, 2049, 2048, 2048, 2048, 2048]",4
7,1239,9550,81,316,229,307,2024-06-30T13:07:51.716000+00:00,70.607,False,17.340,31.678,21.589,"[None, 2049, 2048, 2048, 2048, 2049]","[2048, 2048, 2048, 2049, 2048, 2048, 2049]","[2048, 2048, 2049, 2048, 2048, 2048, 2049]",5
8,1239,9550,81,318,226,308,2024-06-30T13:05:30.092000+00:00,70.743,False,17.362,31.679,21.702,"[None, 2049, 2049, 2048, 2048, 2049]","[2048, 2049, 2048, 2049, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2049, 2048, 2048]",3
9,1239,9550,81,294,225,293,2024-06-30T13:22:06.795000+00:00,71.535,False,17.863,31.835,21.837,"[None, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048]",17


In [13]:
for driver_number, df in driver_arrays.items():
    df.to_csv(f"../Data/lap_data_{driver_number}.csv", index=False)

Interval Data

In [14]:
session_key = 9550
intervals_df = []

for interval in np.arange(0.01, 2.0, 0.01):  
    interval_df = get_intervals(session_key, interval)
    intervals_df.append(interval_df)
    time.sleep(1)

final_intervals_df = pd.concat(intervals_df, ignore_index=True)
final_intervals_df

,session_key,meeting_key,date,driver_number,gap_to_leader,interval
0,9550,1239,2024-06-30T12:07:38.634000+00:00,1,0.0,0.000
1,9550,1239,2024-06-30T13:03:09.919000+00:00,20,1.361,0.001
2,9550,1239,2024-06-30T13:03:19.773000+00:00,3,2.821,0.005
3,9550,1239,2024-06-30T13:03:48.656000+00:00,10,4.645,0.008
4,9550,1239,2024-06-30T13:04:14.393000+00:00,1,0.0,0.000
...,...,...,...,...,...,...
992319,9550,1239,2024-06-30T14:28:26.583000+00:00,31,67.85,0.857
992320,9550,1239,2024-06-30T14:28:26.967000+00:00,10,61.766,0.597
992321,9550,1239,2024-06-30T14:28:27.775000+00:00,31,68.404,1.248
992322,9550,1239,2024-06-30T14:28:30.646000+00:00,31,68.345,1.264


In [15]:
final_intervals_df.to_csv("../Data/interval_data.csv", index=False)

Position Data

In [16]:
meeting_key = 1239
final_position_dfs = []

for driver_num in driver_numbers:
    driver_position_dfs = []  
    for position in range(driver_numbers.size):
        position_df = get_position(meeting_key, driver_num, position)
        driver_position_dfs.append(position_df)
        time.sleep(1)
        
    driver_df_concat = pd.concat(driver_position_dfs, ignore_index=True)
    final_position_dfs.append(driver_df_concat)

final_position_df = pd.concat(final_position_dfs, ignore_index=True)
final_position_df

,session_key,meeting_key,driver_number,date,position
0,9540,1239,1,2024-06-28T10:16:53.192000+00:00,1
1,9540,1239,1,2024-06-28T10:35:34.221000+00:00,1
2,9540,1239,1,2024-06-28T10:40:24.265000+00:00,1
3,9540,1239,1,2024-06-28T11:25:18.678000+00:00,1
4,9545,1239,1,2024-06-28T14:19:26.696000+00:00,1
...,...,...,...,...,...
32918,9550,1239,81,2024-06-30T14:04:59.079000+00:00,5
32919,9550,1239,81,2024-06-30T14:05:03.959000+00:00,6
32920,9550,1239,81,2024-06-30T14:07:28.131000+00:00,5
32921,9550,1239,81,2024-06-30T14:19:02.566000+00:00,3


In [23]:
final_position_df.to_csv("../Data/position_data.csv", index=False)

Pit Data

In [18]:
session_key = 9550
pit_df = get_pit_data(session_key)
pit_df

,session_key,meeting_key,date,driver_number,pit_duration,lap_number
0,9550,1239,2024-06-30T13:04:53.204000+00:00,16,31.9,1
1,9550,1239,2024-06-30T13:04:55.523000+00:00,2,31.7,1
2,9550,1239,2024-06-30T13:15:24.903000+00:00,20,21.5,10
3,9550,1239,2024-06-30T13:15:25.998000+00:00,3,21.5,10
4,9550,1239,2024-06-30T13:16:35.747000+00:00,27,22.3,11
5,9550,1239,2024-06-30T13:16:40.741000+00:00,14,21.4,11
6,9550,1239,2024-06-30T13:17:53.027000+00:00,23,21.8,12
7,9550,1239,2024-06-30T13:23:00.965000+00:00,16,21.6,16
8,9550,1239,2024-06-30T13:26:14.563000+00:00,31,22.2,19
9,9550,1239,2024-06-30T13:26:23.447000+00:00,77,21.9,19


In [25]:
pit_df.to_csv("../Data/pit_data.csv", index=False)

Stint Data

In [20]:
session_key = 9550
stint_df = get_stints(session_key)
stint_df

,meeting_key,session_key,stint_number,driver_number,lap_start,lap_end,compound,tyre_age_at_start
0,1239,9550,1,2,1,1,MEDIUM,0
1,1239,9550,1,16,1,1,MEDIUM,0
2,1239,9550,1,3,1,10,MEDIUM,0
3,1239,9550,1,20,1,10,MEDIUM,0
4,1239,9550,1,14,1,11,MEDIUM,3
...,...,...,...,...,...,...,...,...
60,1239,9550,5,16,52,72,MEDIUM,6
61,1239,9550,3,81,52,72,MEDIUM,0
62,1239,9550,3,44,54,72,MEDIUM,0
63,1239,9550,4,1,65,72,SOFT,0


In [27]:
stint_df.to_csv("../Data/stint_data.csv", index=False)